In [1]:
import boto3
from langchain.llms import Bedrock
from langchain.utilities.anthropic import get_num_tokens_anthropic

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain_token_usage.handlers import LocalTokenUsageCallbackHandler
from langchain_token_usage.reporters import LocalStatsReporter, CloudWatchTokenUsageReporter

In [2]:
MODEL_NAME = "anthropic.claude-v2"

def bedrock_anthropic_claude_cost_func(num_input_tokens: int, num_output_tokens: int) -> float:
    """See https://aws.amazon.com/bedrock/pricing/"""
    return num_input_tokens * 0.01102 / 1000.0 + num_output_tokens * 0.03268 / 1000.0

reporter1 = LocalStatsReporter()
reporter2 = CloudWatchTokenUsageReporter(
    namespace="bedrock_token_usage",
    dimensions={"project": "toke_usage_test"}
)

handler = LocalTokenUsageCallbackHandler(
    reporter=reporter2,
    model_name=MODEL_NAME,
    caller_id="test_user",
    token_counter_func=get_num_tokens_anthropic,
    cost_func=bedrock_anthropic_claude_cost_func,
)

session = boto3.Session(region_name="us-east-1")
llm = Bedrock(
    client=session.client("bedrock-runtime"),
    model_id=MODEL_NAME,
    model_kwargs={
        "max_tokens_to_sample": 4096,
        "temperature": 0.2,
    },
    callbacks=[handler],
)

prompt = PromptTemplate.from_template("What city is the capital of {country}?")

chain = LLMChain(llm=llm, prompt=prompt)
chain.run(country="Italy")

[{'Dimensions': [{'Name': 'project', 'Value': 'toke_usage_test'},
                 {'Name': 'model_name', 'Value': 'anthropic.claude-v2'},
                 {'Name': 'caller_id', 'Value': 'test_user'}],
  'MetricName': 'completion_tokens',
  'Timestamp': datetime.datetime(2023, 11, 9, 17, 50, 26, 659150, tzinfo=datetime.timezone.utc),
  'Unit': 'Count',
  'Value': 10.0},
 {'Dimensions': [{'Name': 'project', 'Value': 'toke_usage_test'},
                 {'Name': 'model_name', 'Value': 'anthropic.claude-v2'},
                 {'Name': 'caller_id', 'Value': 'test_user'}],
  'MetricName': 'prompt_tokens',
  'Timestamp': datetime.datetime(2023, 11, 9, 17, 50, 26, 659150, tzinfo=datetime.timezone.utc),
  'Unit': 'Count',
  'Value': 8.0},
 {'Dimensions': [{'Name': 'project', 'Value': 'toke_usage_test'},
                 {'Name': 'model_name', 'Value': 'anthropic.claude-v2'},
                 {'Name': 'caller_id', 'Value': 'test_user'}],
  'MetricName': 'total_tokens',
  'Timestamp': datetime.d

' Rome is the capital and largest city of Italy.'

In [3]:
# reporter1